In [1]:
import pandas as pd
import numpy as np

from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

from preprocessing import preprocesamiento_basico
from graficos_modelos import mostrar_reporte_clasificacion, graficar_auc_roc,graficar_matriz_confusion
from funciones_auxiliares import traer_datasets, traer_dataset_prediccion_final, separar_dataset, encontrar_hiperparametros_RGSCV, mapear_target_binario

# Random Forest

### Obtención de los datos y preprocesamiento

In [ ]:
df, df_sin_target, solo_target = traer_datasets()

X_train, X_test, y_train, y_test = separar_dataset(df_sin_target, solo_target)

X_train.is_copy=False
X_test.is_copy=False
y_train.is_copy=False
y_test.is_copy=False

y_train.set_index('id', inplace=True)
y_train = y_train.sort_values(by=['id'], ascending=True).copy()

y_test.set_index('id', inplace=True)
y_test = y_test.sort_values(by=['id'], ascending=True).copy()


Aplicamos metodo de feature engineering general

In [ ]:
dataframes_procesados = preprocesamiento_basico([X_train, X_test])
X_train = dataframes_procesados[0]
X_test = dataframes_procesados[1]

### Búsqueda de Hiperparámetros

Buscamos hiperparámetros con RandomGridSearch, pues GridSearch iterativamente es bastante lento.

In [ ]:
params = {'criterion': ['gini','entropy'], 'n_estimators':[100,300,500],'max_depth': np.arange(3,12)}
hiperparametros = encontrar_hiperparametros_RGSCV(RandomForestClassifier(), params=params, x_np=X_train.to_numpy(), y_np=y_train.to_numpy())

In [ ]:
criterio_elegido = hiperparametros['criterion']
cantidad_estimadores_elegida = hiperparametros['n_estimators']
profundidad_elegida = hiperparametros['max_depth']
print(f'Mejor criterio: {criterio_elegido}')
print(f'Mejor cantidad de estimadores: {cantidad_estimadores_elegida}')
print(f'Mejor profundidad_maxima: {profundidad_elegida}')

### Entrenando el Modelo con CrossValidation

Procedemos a testear con kfolds, stratificados pues nuestro dataset es desbalanceado. Además, usamos los hiperparámetros encontrados previamente.

In [ ]:
kf = StratifiedKFold(n_splits=5)
for fold_idx, (train_index, test_index) in enumerate(kf.split(X_train, y_train)):
    bosque_clasificacion = RandomForestClassifier(max_depth=profundidad_elegida, n_estimators=cantidad_estimadores_elegida, criterion=criterio_elegido)
    bosque_clasificacion.fit(X_train.iloc[train_index], y_train.iloc[train_index])
    print ('Reporte para el FOLD ' + str(fold_idx))
    print(classification_report(y_train.iloc[test_index], bosque_clasificacion.predict(X_train.iloc[test_index]), target_names=['No llueven hamburguesas al dia siguiente', 'Llueven hamburguesas al dia siguiente']))
    

### Predicción del modelo sobre holdout

Una vez entrenado y validado el modelo con CrossValidation, viendo que las métricas resultantes fueron razonables y no dan indicio de overfit o mala división del dataset, pasaremos a probar el modelo en la partición Holdout. Para esto usaremos los mejores hiperparámetros hallados previamente en Training.

In [ ]:
bosque_holdout = RandomForestClassifier(max_depth=profundidad_elegida, n_estimators=cantidad_estimadores_elegida, criterion=criterio_elegido)
bosque_holdout.fit(X_train.to_numpy(), y_train.to_numpy())

In [ ]:
mapeo_binario_v = np.vectorize(mapear_target_binario)
y_pred = bosque_holdout.predict(X_test.to_numpy())
y_pred_binario = mapeo_binario_v(y_pred)
y_pred_proba = bosque_holdout.predict_proba(X_test.to_numpy())[:, 1]
y_test_binario = y_test['llovieron_hamburguesas_al_dia_siguiente'].map({'si': 1, 'no': 0}).to_numpy()
print(classification_report(y_test['llovieron_hamburguesas_al_dia_siguiente'].to_numpy(), y_pred))

### Curva AUC ROC


In [ ]:
graficar_auc_roc(y_test_binario, y_pred_proba)

### Matriz de confusion

In [ ]:
graficar_matriz_confusion(y_test_binario, y_pred_binario)

## Predicción con el dataset nuevo
A continuación, realizamos la predicción con el RandomForest sobre el dataset de predicciones nuevo, y la escribimos al archivo 'RandomForest.csv'

In [ ]:
from funciones_auxiliares import exportar_prediccion_final

df_prediccion_final = traer_dataset_prediccion_final()
ids = df_prediccion_final['id'].to_numpy()

df_prediccion_final = preprocesamiento_basico([df_prediccion_final])[0]
predicciones = bosque_holdout.predict(df_prediccion_final.to_numpy())

exportar_prediccion_final(ids, predicciones, 'RandomForest')